# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [3]:
from bs4 import BeautifulSoup
import requests
import re
import time
import threading
import json
from urllib.parse import urljoin

In [4]:
PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

In [15]:
# 爬取文章
def crawl_article(url):
    response = requests.get(url, cookies={'over18': '1'})
    
    # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
    if response.status_code != 200:
        print('Error - {} is not available to access'.format(url))
        return
    
    # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
    soup = BeautifulSoup(response.text)
    
    # 取得文章內容主體
    main_content = soup.find(id='main-content')
    
    # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
    metas = main_content.select('div.article-metaline')
    author = ''
    title = ''
    date = ''
    if metas:
        if metas[0].select('span.article-meta-value')[0]:
            author = metas[0].select('span.article-meta-value')[0].string # author
        if metas[1].select('span.article-meta-value')[0]:
            title = metas[1].select('span.article-meta-value')[0].string # title
        if metas[2].select('span.article-meta-value')[0]:
            date = metas[2].select('span.article-meta-value')[0].string # date

        # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
        #
        # .extract() 方法可以參考官方文件
        #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
        for m in metas:
            m.extract()
        for m in main_content.select('div.article-metaline-right'):
            m.extract()
    
    # 取得留言區主體
    pushes = main_content.find_all('div', class_='push')
    for p in pushes:
        p.extract()
    
    # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
    # 透過 regular expression 取得 IP
    # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
    try:
        ip = main_content.find(text=re.compile(u'※ 發信站:'))
        ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
    except Exception as e:
        ip = ''
    
    # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
    # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
    #
    # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
    #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
    filtered = []
    for v in main_content.stripped_strings:
        # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
        if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
            filtered.append(v)

    # 定義一些特殊符號與全形符號的過濾器
    expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
    for i in range(len(filtered)):
        filtered[i] = re.sub(expr, '', filtered[i])
    
    # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
    filtered = [i for i in filtered if i]
    content = ' '.join(filtered)
    
    # 處理留言區
    # p 計算推文數量
    # b 計算噓文數量
    # n 計算箭頭數量
    p, b, n = 0, 0, 0
    messages = []
    for push in pushes:
        # 假如留言段落沒有 push-tag 就跳過
        if not push.find('span', 'push-tag'):
            continue
        
        # 過濾額外空白與換行符號
        # push_tag 判斷是推文, 箭頭還是噓文
        # push_userid 判斷留言的人是誰
        # push_content 判斷留言內容
        # push_ipdatetime 判斷留言日期時間
        push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
        push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
        push_content = push.find('span', 'push-content').strings
        push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
        push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

        # 整理打包留言的資訊, 並統計推噓文數量
        messages.append({
            'push_tag': push_tag,
            'push_userid': push_userid,
            'push_content': push_content,
            'push_ipdatetime': push_ipdatetime})
        if push_tag == u'推':
            p += 1
        elif push_tag == u'噓':
            b += 1
        else:
            n += 1
    
    # 統計推噓文
    # count 為推噓文相抵看這篇文章推文還是噓文比較多
    # all 為總共留言數量 
    message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}
    
    # 整理文章資訊
    data = {
        'url': url,
        'article_author': author,
        'article_title': title,
        'article_date': date,
        'article_content': content,
        'ip': ip,
        'message_count': message_count,
        'messages': messages
    }
    return data

In [16]:
import time

# 對文章列表送出請求並取得列表主體
resp = requests.get(PTT_URL, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)
main_list = soup.find('div', class_='bbs-screen')
all_data = []

stime = time.time()
# 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
for div in main_list.findChildren('div', recursive=False):
    class_name = div.attrs['class']  #['search-bar']['r-ent']['r-ent']...
    
    # 遇到分隔線要處理的情況
    if class_name and 'r-list-sep' in class_name:
        print('Reach the last article')
        break
    
    # 遇到目標文章
    if class_name and 'r-ent' in class_name:
        div_title = div.find('div', class_='title')
        a_title = div_title.find('a', href=True)
        if a_title:
            article_URL = urljoin(PTT_URL, a_title['href'])
        else:
            article_URL = None
            a_title = '<a>本文已刪除</a>'
        article_title = a_title.text
        print('Parse {} - {}'.format(article_title, article_URL))
        
        # 呼叫上面寫好的 function 來對文章進行爬蟲
        if article_URL:
            parse_data = crawl_article(article_URL) # 返回單一文章資訊的字典
        
        # 將爬完的資料儲存
        all_data.append(parse_data)
        
etime = time.time()
print('共用時：',etime-stime )

Parse Re: [爆卦] 中國網軍在蒐集台灣身分證 - https://www.ptt.cc/bbs/Gossiping/M.1587092228.A.D55.html
Parse Re: [問卦] 政治系大一就教台獨嗎？ - https://www.ptt.cc/bbs/Gossiping/M.1587092326.A.D84.html
Parse Re: [政治] 華航護照更名公決案今立院闖關　民進黨 - https://www.ptt.cc/bbs/Gossiping/M.1587092359.A.E4E.html
Parse [問卦] 年薪不到百萬的都是什麼職業 - https://www.ptt.cc/bbs/Gossiping/M.1587092366.A.38D.html
Parse [問卦] FF7ac在當年(2005)算是黑科技動畫嗎 - https://www.ptt.cc/bbs/Gossiping/M.1587092444.A.9A2.html
Parse [政治] 受困莫三比克生命受威脅 自稱台灣男子上 - https://www.ptt.cc/bbs/Gossiping/M.1587092449.A.E77.html
Parse Re: [新聞] 10多年來堅持不喝牛奶！台灣糖尿病之父 - https://www.ptt.cc/bbs/Gossiping/M.1587092454.A.0F8.html
Parse [問卦] 台積電是在噴幾點的啦！ - https://www.ptt.cc/bbs/Gossiping/M.1587092497.A.3B8.html
Parse [爆卦] 垃圾三立新聞造假移植圖片又一例!! - https://www.ptt.cc/bbs/Gossiping/M.1587092499.A.00C.html
Parse Re: [問卦] 國中基測是不是比較能看出讀書天賦？ - https://www.ptt.cc/bbs/Gossiping/M.1587092523.A.D5C.html
Parse Re: [問卦] 你們通通給我尊重林奕含!! - https://www.ptt.cc/bbs/Gossiping/M.1587092527.A.838.html
Parse Re: [新聞] 小五女生被狗追趕

In [17]:
parse_data

{'url': 'https://www.ptt.cc/bbs/Gossiping/M.1587092707.A.6EF.html',
 'article_author': 'palindromes ()',
 'article_title': '[問卦] 怎樣才能開一家觀光客在吃的店？',
 'article_date': 'Fri Apr 17 11:05:05 2020',
 'article_content': '我發現一堆觀光客的店\n\n明明很難吃卻能大排長龍\n\n詭異程度就如八卦某些爆文\n\n沒質還能推到爆這樣\n\n\n奇怪的是\n\n本地人不愛\n\n外地人吃了一次就不去\n\n但總是人排 有人買\n\n這就是觀光客在吃的店厲害的地方\n\n那麼問題來了\n\n要想成為一家觀光客在吃的店\n\n到底要怎麼做呢？\n\n\n有沒有八卦？\n\n\n\n\n\n\n\n\nhttps://www.ptt.cc/bbs/Gossiping/M.1587092707.A.6EF.html',
 'ip': '111.83.36.14',
 'message_count': {'all': 0, 'count': 0, 'push': 0, 'boo': 0, 'neutral': 0},
 'messages': []}

### **多線程爬蟲**

In [18]:
import time

# 對文章列表送出請求並取得列表主體
resp = requests.get(PTT_URL, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)
main_list = soup.find('div', class_='bbs-screen')
all_data = []
all_url = []

stime = time.time()
# 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
for div in main_list.findChildren('div', recursive=False):
    class_name = div.attrs['class']  #['search-bar']['r-ent']['r-ent']...
    
    # 遇到分隔線要處理的情況
    if class_name and 'r-list-sep' in class_name:
        print('Reach the last article')
        break
    
    # 遇到目標文章
    if class_name and 'r-ent' in class_name:
        div_title = div.find('div', class_='title')
        a_title = div_title.find('a', href=True)
        if a_title:
            article_URL = urljoin(PTT_URL, a_title['href'])
        else:
            article_URL = None
            a_title = '<a>本文已刪除</a>'
        article_title = a_title.text
        print('Parse {} - {}'.format(article_title, article_URL))
        
        # 把文章連結存在list
        if article_URL:
            all_url.append(article_URL)

# 從這裡丟給子執行緒工作            
# 建立 n 個子執行緒，分別去抓文章內容
threads = []
for i in range(len(all_url)):
    threads.append(threading.Thread(target = crawl_article, args = (all_url[i],)))
    threads[i].start()

# 主執行緒繼續執行自己的工作
# ...

# 等待所有子執行緒結束
for i in range(len(all_url)):
    threads[i].join()

        
etime = time.time()
print('共用時：',etime-stime )

Parse [新聞] 已婚女醫健身愛上大肌肌,瞞醫師夫偷生娃. - https://www.ptt.cc/bbs/Gossiping/M.1587092710.A.176.html
Parse [爆卦] <Nature>公開向汙名化武漢肺炎公開道歉!! - https://www.ptt.cc/bbs/Gossiping/M.1587092722.A.0F6.html
Parse Re: [新聞] 超車自撞牽拖別人竟連孕婦都打！9屁孩國 - https://www.ptt.cc/bbs/Gossiping/M.1587092734.A.E59.html
Parse [問卦] 打算騎Gogoro去司馬庫斯要怎麼計劃 - https://www.ptt.cc/bbs/Gossiping/M.1587092749.A.1F5.html
Parse [問卦] 藥局有強制加入健保藥局嗎？ - https://www.ptt.cc/bbs/Gossiping/M.1587092782.A.C0D.html
Parse [新聞] 女兒載閨密遭酒駕撞　母路口舉牌求援… - https://www.ptt.cc/bbs/Gossiping/M.1587092862.A.0B3.html
Parse Re: [問卦] 歐巴馬的健保 是有多爛???? - https://www.ptt.cc/bbs/Gossiping/M.1587092876.A.3F6.html
Reach the last article
共用時： 0.12828803062438965


### **Class**

In [20]:
class MyTask(threading.Thread):
    def __init__(self, task_name):
        super(MyTask, self).__init__()
        self.task_name = task_name

    def run(self):
        print("Get task: {}\n".format(self.task_name))
        time.sleep(1)
        print("Finish task: {}\n".format(self.task_name))



if __name__ == "__main__":
    data = [1,2,3,4,5,6,7,8,9,10]
    tasks = []
    for i in range(0, 10):
        # 建立 task
        tasks.append(MyTask("task_{}".format(data[i])))
    for t in tasks:
        # 開始執行 task
        t.start()

    for t in tasks:
        # 等待 task 執行完畢
        # 完畢前會阻塞住主執行緒
        t.join()
    print("Finish.")

In [21]:
PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

class Crawl_Article(threading.Thread):
    
    def __init__(self, url):
        super(Crawl_Article, self).__init__()
        self.url = url

    # 原crawl_article，改成子執行緒run任務
    def run(self): 
        print("Get子執行緒: {}\n".format(self.url))

        response = requests.get(self.url, cookies={'over18': '1'})

        # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
        if response.status_code != 200:
            print('Error - {} is not available to access'.format(self.url))
            return

        # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
        soup = BeautifulSoup(response.text)

        # 取得文章內容主體
        main_content = soup.find(id='main-content')

        # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
        metas = main_content.select('div.article-metaline') #list
        author = ''
        title = ''
        date = ''
        if metas:
            if metas[0].select('span.article-meta-value')[0]:
                author = metas[0].select('span.article-meta-value')[0].string
            if metas[1].select('span.article-meta-value')[0]:
                title = metas[1].select('span.article-meta-value')[0].string
            if metas[2].select('span.article-meta-value')[0]:
                date = metas[2].select('span.article-meta-value')[0].string

            # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
            #
            # .extract() 方法可以參考官方文件
            #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
            for m in metas:
                m.extract()
            for m in main_content.select('div.article-metaline-right'):
                m.extract()

        # 取得留言區主體
        pushes = main_content.find_all('div', class_='push')
        for p in pushes:
            p.extract()

        # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
        # 透過 regular expression 取得 IP
        # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
        try:
            ip = main_content.find(text=re.compile(u'※ 發信站:'))
            ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
        except Exception as e:
            ip = ''

        # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
        # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
        #
        # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
        #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
        filtered = []
        for v in main_content.stripped_strings:
            # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
            if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
                filtered.append(v)

        # 定義一些特殊符號與全形符號的過濾器
        expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
        for i in range(len(filtered)):
            filtered[i] = re.sub(expr, '', filtered[i])

        # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
        filtered = [i for i in filtered if i]
        content = ' '.join(filtered)

        # 處理留言區
        # p 計算推文數量
        # b 計算噓文數量
        # n 計算箭頭數量
        p, b, n = 0, 0, 0
        messages = []
        for push in pushes:
            # 假如留言段落沒有 push-tag 就跳過
            if not push.find('span', 'push-tag'):
                continue

            # 過濾額外空白與換行符號
            # push_tag 判斷是推文, 箭頭還是噓文
            # push_userid 判斷留言的人是誰
            # push_content 判斷留言內容
            # push_ipdatetime 判斷留言日期時間
            push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
            push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
            push_content = push.find('span', 'push-content').strings
            push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
            push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

            # 整理打包留言的資訊, 並統計推噓文數量
            messages.append({
                'push_tag': push_tag,
                'push_userid': push_userid,
                'push_content': push_content,
                'push_ipdatetime': push_ipdatetime})
            if push_tag == u'推':
                p += 1
            elif push_tag == u'噓':
                b += 1
            else:
                n += 1

        # 統計推噓文
        # count 為推噓文相抵看這篇文章推文還是噓文比較多
        # all 為總共留言數量 
        message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}

        # 整理文章資訊
        data = {
            'url': self.url,
            'article_author': author,
            'article_title': title,
            'article_date': date,
            'article_content': content,
            'ip': ip,
            'message_count': message_count,
            'messages': messages
        }
        return data

import time

if __name__ == '__main__':
    
    # 對文章列表送出請求並取得列表主體
    resp = requests.get(PTT_URL, cookies={'over18': '1'})
    soup = BeautifulSoup(resp.text)
    main_list = soup.find('div', class_='bbs-screen')
    all_data = []
    all_url = []

    stime = time.time()
    # 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
    for div in main_list.findChildren('div', recursive=False):
        class_name = div.attrs['class']  #['search-bar']['r-ent']['r-ent']...
        # 遇到分隔線要處理的情況
        if class_name and 'r-list-sep' in class_name:
            print('Reach the last article')
            break
        # 遇到目標文章
        if class_name and 'r-ent' in class_name:
            div_title = div.find('div', class_='title')
            a_title = div_title.find('a', href=True)
            if a_title:
                article_URL = urljoin(PTT_URL, a_title['href'])
            else:
                article_URL = None
                a_title = '<a>本文已刪除</a>'
            article_title = a_title.text
            print('Parse {} - {}'.format(article_title, article_URL))
            # 把文章連結存在list
            if article_URL:
                all_url.append(article_URL)
    
    print('共{}個連結'.format(len(all_url)))
    # 從這裡丟給子執行緒工作            
    # 建立 n 個子執行緒，分別去抓文章內容
    threads = []
    for i in range(len(all_url)):
        threads.append(Crawl_Article(all_url[i]))
        threads[i].start()

    # 主執行緒繼續執行自己的工作
    # ...

    # 等待所有子執行緒結束
    for i in range(len(all_url)):
        threads[i].join()


    etime = time.time()
    print('共用時：',etime-stime )

Parse [新聞] 已婚女醫健身愛上大肌肌,瞞醫師夫偷生娃. - https://www.ptt.cc/bbs/Gossiping/M.1587092710.A.176.html
Parse [爆卦] <Nature>公開向汙名化武漢肺炎公開道歉!! - https://www.ptt.cc/bbs/Gossiping/M.1587092722.A.0F6.html
Parse Re: [新聞] 超車自撞牽拖別人竟連孕婦都打！9屁孩國 - https://www.ptt.cc/bbs/Gossiping/M.1587092734.A.E59.html
Parse [問卦] 打算騎Gogoro去司馬庫斯要怎麼計劃 - https://www.ptt.cc/bbs/Gossiping/M.1587092749.A.1F5.html
Parse [問卦] 藥局有強制加入健保藥局嗎？ - https://www.ptt.cc/bbs/Gossiping/M.1587092782.A.C0D.html
Parse [新聞] 女兒載閨密遭酒駕撞　母路口舉牌求援… - https://www.ptt.cc/bbs/Gossiping/M.1587092862.A.0B3.html
Parse Re: [問卦] 歐巴馬的健保 是有多爛???? - https://www.ptt.cc/bbs/Gossiping/M.1587092876.A.3F6.html
Parse Re: [政治] 華航護照更名公決案今立院闖關　民進黨 - https://www.ptt.cc/bbs/Gossiping/M.1587092893.A.245.html
Parse Re: [問卦] 今天請假 大台北 哪裡可以逛 ? - https://www.ptt.cc/bbs/Gossiping/M.1587092937.A.B30.html
Reach the last article
共9個連結
Get子執行緒: https://www.ptt.cc/bbs/Gossiping/M.1587092710.A.176.html

Get子執行緒: https://www.ptt.cc/bbs/Gossiping/M.1587092722.A.0F6.html

Get子執行緒: https://www

### **使用佇列 Queue**

In [22]:
from queue import Queue

PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

class Crawl_Article(threading.Thread):
    
    def __init__(self, queue):
        super(Crawl_Article, self).__init__()
        self.queue = queue

    # 原crawl_article，改成子執行緒run任務
    def run(self): 
        # 當 queue 裡面有資料再執行
        while self.queue.qsize() > 0:
            url = self.queue.get()
            print("Get子執行緒: {}\n".format(url))

            response = requests.get(url, cookies={'over18': '1'})

            # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
            if response.status_code != 200:
                print('Error - {} is not available to access'.format(url))
                return

            # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
            soup = BeautifulSoup(response.text)

            # 取得文章內容主體
            main_content = soup.find(id='main-content')

            # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
            metas = main_content.select('div.article-metaline') #list
            author = ''
            title = ''
            date = ''
            if metas:
                if metas[0].select('span.article-meta-value')[0]:
                    author = metas[0].select('span.article-meta-value')[0].string
                if metas[1].select('span.article-meta-value')[0]:
                    title = metas[1].select('span.article-meta-value')[0].string
                if metas[2].select('span.article-meta-value')[0]:
                    date = metas[2].select('span.article-meta-value')[0].string

                # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
                #
                # .extract() 方法可以參考官方文件
                #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
                for m in metas:
                    m.extract()
                for m in main_content.select('div.article-metaline-right'):
                    m.extract()

            # 取得留言區主體
            pushes = main_content.find_all('div', class_='push')
            for p in pushes:
                p.extract()

            # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
            # 透過 regular expression 取得 IP
            # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
            try:
                ip = main_content.find(text=re.compile(u'※ 發信站:'))
                ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
            except Exception as e:
                ip = ''

            # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
            # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
            #
            # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
            #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
            filtered = []
            for v in main_content.stripped_strings:
                # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
                if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
                    filtered.append(v)

            # 定義一些特殊符號與全形符號的過濾器
            expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
            for i in range(len(filtered)):
                filtered[i] = re.sub(expr, '', filtered[i])

            # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
            filtered = [i for i in filtered if i]
            content = ' '.join(filtered)

            # 處理留言區
            # p 計算推文數量
            # b 計算噓文數量
            # n 計算箭頭數量
            p, b, n = 0, 0, 0
            messages = []
            for push in pushes:
                # 假如留言段落沒有 push-tag 就跳過
                if not push.find('span', 'push-tag'):
                    continue

                # 過濾額外空白與換行符號
                # push_tag 判斷是推文, 箭頭還是噓文
                # push_userid 判斷留言的人是誰
                # push_content 判斷留言內容
                # push_ipdatetime 判斷留言日期時間
                push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
                push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
                push_content = push.find('span', 'push-content').strings
                push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
                push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

                # 整理打包留言的資訊, 並統計推噓文數量
                messages.append({
                    'push_tag': push_tag,
                    'push_userid': push_userid,
                    'push_content': push_content,
                    'push_ipdatetime': push_ipdatetime})
                if push_tag == u'推':
                    p += 1
                elif push_tag == u'噓':
                    b += 1
                else:
                    n += 1

            # 統計推噓文
            # count 為推噓文相抵看這篇文章推文還是噓文比較多
            # all 為總共留言數量 
            message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}

            # 整理文章資訊
            data = {
                'url': url,
                'article_author': author,
                'article_title': title,
                'article_date': date,
                'article_content': content,
                'ip': ip,
                'message_count': message_count,
                'messages': messages
            }
            return data

import time

if __name__ == '__main__':
    
    # 對文章列表送出請求並取得列表主體
    resp = requests.get(PTT_URL, cookies={'over18': '1'})
    soup = BeautifulSoup(resp.text)
    main_list = soup.find('div', class_='bbs-screen')
    all_data = []
    
    Q_url = Queue()

    stime = time.time()
    # 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
    for div in main_list.findChildren('div', recursive=False):
        class_name = div.attrs['class']  #['search-bar']['r-ent']['r-ent']...
        # 遇到分隔線要處理的情況
        if class_name and 'r-list-sep' in class_name:
            print('Reach the last article')
            break
        # 遇到目標文章
        if class_name and 'r-ent' in class_name:
            div_title = div.find('div', class_='title')
            a_title = div_title.find('a', href=True)
            if a_title:
                article_URL = urljoin(PTT_URL, a_title['href'])
            else:
                article_URL = None
                a_title = '<a>本文已刪除</a>'
            article_title = a_title.text
            print('Parse {} - {}'.format(article_title, article_URL))
            # 把文章連結存在list
            if article_URL:
                Q_url.put(article_URL)
    
    print('共{}個連結'.format(Q_url.qsize()))
    
    # 從這裡丟給子執行緒工作            
    # 建立 n 個子執行緒，分別去抓文章內容
    threads = []
    for i in range(Q_url.qsize()):
        threads.append(Crawl_Article(Q_url))
        threads[i].start()

    # 主執行緒繼續執行自己的工作
    # ...

    # 等待所有子執行緒結束
    for i in range(len(all_url)):
        threads[i].join()


    etime = time.time()
    print('共用時：',etime-stime )

Parse [新聞] 已婚女醫健身愛上大肌肌,瞞醫師夫偷生娃. - https://www.ptt.cc/bbs/Gossiping/M.1587092710.A.176.html
Parse [爆卦] <Nature>公開向汙名化武漢肺炎公開道歉!! - https://www.ptt.cc/bbs/Gossiping/M.1587092722.A.0F6.html
Parse Re: [新聞] 超車自撞牽拖別人竟連孕婦都打！9屁孩國 - https://www.ptt.cc/bbs/Gossiping/M.1587092734.A.E59.html
Parse [問卦] 打算騎Gogoro去司馬庫斯要怎麼計劃 - https://www.ptt.cc/bbs/Gossiping/M.1587092749.A.1F5.html
Parse [問卦] 藥局有強制加入健保藥局嗎？ - https://www.ptt.cc/bbs/Gossiping/M.1587092782.A.C0D.html
Parse [新聞] 女兒載閨密遭酒駕撞　母路口舉牌求援… - https://www.ptt.cc/bbs/Gossiping/M.1587092862.A.0B3.html
Parse Re: [問卦] 歐巴馬的健保 是有多爛???? - https://www.ptt.cc/bbs/Gossiping/M.1587092876.A.3F6.html
Parse Re: [政治] 華航護照更名公決案今立院闖關　民進黨 - https://www.ptt.cc/bbs/Gossiping/M.1587092893.A.245.html
Parse Re: [問卦] 今天請假 大台北 哪裡可以逛 ? - https://www.ptt.cc/bbs/Gossiping/M.1587092937.A.B30.html
Parse [問卦] 有沒有商家拿 0 確診做促銷的八卦 - https://www.ptt.cc/bbs/Gossiping/M.1587092946.A.64D.html
Parse Re: [問卦] 美國航母不過是東風21D的靶船不是嗎?? - https://www.ptt.cc/bbs/Gossiping/M.1587092986.A.B8A.html


### **使用lock:**
* 被 Lock 的 acquire 與 release 包起來的這段程式碼不會被兩個執行緒同時執行。
* 用來寫入檔案

In [23]:
class Worker(threading.Thread):
    
    def __init__(self, queue, num, lock):
        
        threading.Thread.__init__(self)
        self.queue = queue
        self.num = num
        self.lock = lock

    def run(self):
        while self.queue.qsize() > 0:
            url = self.queue.get()

            # 取得 lock
            lock.acquire()
            print("子執行緒 %d 取得lock" % self.num)

            # 不能讓多個執行緒同時進的工作
            print("子執行緒 %d: 寫入檔案 %s" % (self.num, url))
            time.sleep(1)

            # 釋放 lock
            print("子執行緒 %d 釋放lock" % self.num)
            self.lock.release()
            
#建立一個佇列
my_queue = Queue()

#假裝放五個URL進去queue
for i in range(1,6):
    my_queue.put("Url %d" % i)

# 建立 lock
lock = threading.Lock()

#建立2個子執行緒，傳入queue和一個參數和lock
my_worker1 = Worker(my_queue, 100, lock)
my_worker2 = Worker(my_queue, 200, lock)

my_worker1.start()
my_worker2.start()

my_worker1.join()
my_worker2.join()

print("Done.")

子執行緒 100 取得lock
子執行緒 100: 寫入檔案 Url 1
子執行緒 100 釋放lock
子執行緒 100 取得lock
子執行緒 100: 寫入檔案 Url 3
子執行緒 100 釋放lock
子執行緒 100 取得lock
子執行緒 100: 寫入檔案 Url 4
子執行緒 100 釋放lock
子執行緒 100 取得lock
子執行緒 100: 寫入檔案 Url 5
子執行緒 100 釋放lock
子執行緒 200 取得lock
子執行緒 200: 寫入檔案 Url 2
子執行緒 200 釋放lock
Done.


In [24]:
from queue import Queue

PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

class Crawl_Article(threading.Thread):
    
    def __init__(self, queue, lock):
        super(Crawl_Article, self).__init__()
        self.queue = queue
        self.lock = lock

    # 原crawl_article，改成子執行緒run任務
    def run(self): 
        # 當 queue 裡面有資料再執行
        while self.queue.qsize() > 0:
            url = self.queue.get()
            #print("Get子執行緒: {}\n".format(url))

            response = requests.get(url, cookies={'over18': '1'})

            # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
            if response.status_code != 200:
                print('Error - {} is not available to access'.format(url))
                return

            # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
            soup = BeautifulSoup(response.text)

            # 取得文章內容主體
            main_content = soup.find(id='main-content')

            # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
            metas = main_content.select('div.article-metaline') #list
            author = ''
            title = ''
            date = ''
            if metas:
                if metas[0].select('span.article-meta-value')[0]:
                    author = metas[0].select('span.article-meta-value')[0].string
                if metas[1].select('span.article-meta-value')[0]:
                    title = metas[1].select('span.article-meta-value')[0].string
                if metas[2].select('span.article-meta-value')[0]:
                    date = metas[2].select('span.article-meta-value')[0].string

                # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
                #
                # .extract() 方法可以參考官方文件
                #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
                for m in metas:
                    m.extract()
                for m in main_content.select('div.article-metaline-right'):
                    m.extract()

            # 取得留言區主體
            pushes = main_content.find_all('div', class_='push')
            for p in pushes:
                p.extract()

            # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
            # 透過 regular expression 取得 IP
            # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
            try:
                ip = main_content.find(text=re.compile(u'※ 發信站:'))
                ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
            except Exception as e:
                ip = ''

            # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
            # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
            #
            # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
            #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
            filtered = []
            for v in main_content.stripped_strings:
                # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
                if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
                    filtered.append(v)

            # 定義一些特殊符號與全形符號的過濾器
            expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
            for i in range(len(filtered)):
                filtered[i] = re.sub(expr, '', filtered[i])

            # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
            filtered = [i for i in filtered if i]
            content = ' '.join(filtered)

            # 處理留言區
            # p 計算推文數量
            # b 計算噓文數量
            # n 計算箭頭數量
            p, b, n = 0, 0, 0
            messages = []
            for push in pushes:
                # 假如留言段落沒有 push-tag 就跳過
                if not push.find('span', 'push-tag'):
                    continue

                # 過濾額外空白與換行符號
                # push_tag 判斷是推文, 箭頭還是噓文
                # push_userid 判斷留言的人是誰
                # push_content 判斷留言內容
                # push_ipdatetime 判斷留言日期時間
                push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
                push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
                push_content = push.find('span', 'push-content').strings
                push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
                push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

                # 整理打包留言的資訊, 並統計推噓文數量
                messages.append({
                    'push_tag': push_tag,
                    'push_userid': push_userid,
                    'push_content': push_content,
                    'push_ipdatetime': push_ipdatetime})
                if push_tag == u'推':
                    p += 1
                elif push_tag == u'噓':
                    b += 1
                else:
                    n += 1

            # 統計推噓文
            # count 為推噓文相抵看這篇文章推文還是噓文比較多
            # all 為總共留言數量 
            message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}

            # 整理文章資訊
            data = {
                'url': url,
                'article_author': author,
                'article_title': title,
                'article_date': date,
                'article_content': content,
                'ip': ip,
                'message_count': message_count,
                'messages': messages
            }
            
            
            # 寫入檔案:單一文章內容
            
            # 取得 lock
            lock.acquire()
            #print("%s 取得lock" % url[32:51])

            # 不能讓多個執行緒同時進的工作 : 將爬完的資訊存成 json 檔案
            #print("寫入檔案")
            with open('../Data/PTT_Article.json', 'a+', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
                f.write(",")

            # 釋放 lock
            #print("%s 釋放lock" % url[32:51])
            self.lock.release()

import time

if __name__ == '__main__':
    
    # 對文章列表送出請求並取得列表主體
    resp = requests.get(PTT_URL, cookies={'over18': '1'})
    soup = BeautifulSoup(resp.text)
    main_list = soup.find('div', class_='bbs-screen')
    all_data = []
    
    Q_url = Queue()

    stime = time.time()
    # 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
    for div in main_list.findChildren('div', recursive=False):
        class_name = div.attrs['class']  #['search-bar']['r-ent']['r-ent']...
        # 遇到分隔線要處理的情況
        if class_name and 'r-list-sep' in class_name:
            print('Reach the last article')
            break
        # 遇到目標文章
        if class_name and 'r-ent' in class_name:
            div_title = div.find('div', class_='title')
            a_title = div_title.find('a', href=True)
            
            if a_title:
                article_URL = urljoin(PTT_URL, a_title['href'])
                article_title = a_title.text
            else:
                article_URL = None
                a_title = '<a>本文已刪除</a>'
                article_title = a_title
                
            #article_title = a_title.text
            print('Parse {} - {}'.format(article_title, article_URL))
            # 把文章連結存在list
            if article_URL:
                Q_url.put(article_URL)
    
    print('共{}個連結'.format(Q_url.qsize()))
    
    # 建立 lock
    lock = threading.Lock()
    
    # 從這裡丟給子執行緒工作            
    # 建立 n 個子執行緒，分別去抓文章內容
    threads = []
    for i in range(Q_url.qsize()):
        threads.append(Crawl_Article(Q_url, lock))
        threads[i].start()

    # 主執行緒繼續執行自己的工作
    # ...

    # 等待所有子執行緒結束
    # for i in range(len(all_url)):
    #     threads[i].join()


    etime = time.time()
    print('共用時：',etime-stime )

Parse Re: [新聞] 已婚女醫健身愛上大肌肌,瞞醫師夫偷生娃. - https://www.ptt.cc/bbs/Gossiping/M.1587093717.A.E9A.html
Parse Re: [政治] 華航護照更名公決案今立院闖關　民進黨 - https://www.ptt.cc/bbs/Gossiping/M.1587093787.A.B5D.html
Parse [問卦] 湘北打山王每看必哭嗎? - https://www.ptt.cc/bbs/Gossiping/M.1587093789.A.143.html
Parse [爆卦] 中國官方訂正武漢市確診病例數和死亡數 - https://www.ptt.cc/bbs/Gossiping/M.1587093796.A.836.html
Parse Re: [新聞] 小五女生被狗追趕狂奔猝死 飼主過失致死 - https://www.ptt.cc/bbs/Gossiping/M.1587093821.A.2AC.html
Parse [政治] 「司馬昭之心全民皆知！」柯建銘批國民黨 - https://www.ptt.cc/bbs/Gossiping/M.1587093822.A.04F.html
Parse Re: [政治] 華航護照更名公決案今立院闖關　民進黨 - https://www.ptt.cc/bbs/Gossiping/M.1587093831.A.606.html
Parse Re: [新聞] 1萬8罰款未繳房子被法拍 婦人跪求無法挽回 - https://www.ptt.cc/bbs/Gossiping/M.1587093885.A.491.html
Parse Re: [爆卦] <Nature>公開向汙名化武漢肺炎公開道歉!! - https://www.ptt.cc/bbs/Gossiping/M.1587093951.A.AB5.html
Parse Re: [問卦] 法律系從大一就教廢死嗎 - https://www.ptt.cc/bbs/Gossiping/M.1587093960.A.A0F.html
Parse [政治] 鄭麗君夫婦存款少3800多萬 仍超過1億元 - https://www.ptt.cc/bbs/Gossiping/M.1587093986.A.

Exception in thread Thread-49:
Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "<ipython-input-24-31d037cd2a71>", line 149, in run
    with open('../Data/PTT_Article.json', 'a+', encoding='utf-8') as f:
FileNotFoundError: [Errno 2] No such file or directory: '../Data/PTT_Article.json'

